# Speech Emotion Recognition, is the act of attempting to recognize human emotion and affective states from speech.

#### Here we use Librosa ,it is a Python package for music and audio analysis. Librosa is basically used when we work with audio data like in music generation, Automatic Speech Recognition

In [1]:
# import required libraries
import librosa
import os  
import soundfile
import glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

#### Define the function to extract important fetures from the audio clip. this function mainly extracts three features :
#### mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound  
#### chroma: Pertains to the 12 different pitch classes 
#### mel: Mel Spectrogram Frequency

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

#### Let’s define a dictionary to hold numbers and the emotions available in the RAVDESS dataset, and a list to hold those we want are : 'CALM' , 'HAPPY' , 'SAD' , 'ANGRY'

In [4]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

emotion_observed=['calm', 'happy','fearful','angry']

#### Load the audio file from the respective path and then identify observed emotions and then extract features 

In [18]:
def load_data(test_size=0.25):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\Shamikarani\\Downloads\\Speech\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in emotion_observed:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [19]:
#Split the data to train and test 
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [20]:
# Shape of the training and test datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [21]:
# Number of feature extracted from audio clip 
print('Features extracted: {x_train.shape[1]}')

Features extracted: {x_train.shape[1]}


In [22]:
x_train

array([[-4.69444733e+02,  2.90473537e+01, -3.08719521e+01, ...,
         1.14944436e-04,  3.68020883e-05,  1.24055823e-05],
       [-5.78170044e+02,  5.42816353e+01,  4.19424200e+00, ...,
         4.20206983e-04,  1.69682709e-04,  5.22450318e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040443e+00, ...,
         4.75216802e-05,  3.46632514e-05,  1.62844426e-05],
       ...,
       [-3.68174896e+02,  1.21515379e+01, -4.14459991e+01, ...,
         5.45278145e-03,  3.56062246e-03,  2.32826290e-03],
       [-4.71690521e+02,  3.68995361e+01, -6.17963743e+00, ...,
         4.36597358e-04,  2.39399364e-04,  1.13478207e-04],
       [-4.19084900e+02,  2.58726368e+01, -2.53285732e+01, ...,
         7.43104727e-04,  6.09643641e-04,  3.89780820e-04]])

#### Build a model using MLPClassifier( Mutli Layered Perceptron ) with best parameters

In [23]:
model=MLPClassifier(alpha=0.03, batch_size=350, epsilon=1e-09, hidden_layer_sizes=(330,), learning_rate='adaptive', max_iter=550,activation='relu')

In [24]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.03, batch_size=350, epsilon=1e-09,
              hidden_layer_sizes=(330,), learning_rate='adaptive',
              max_iter=550)

In [25]:
# Prediction on test data set
y_pred=model.predict(x_test)

In [26]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'fearful', 'calm', 'calm',
       'angry', 'angry', 'happy', 'happy', 'fearful', 'fearful', 'happy',
       'fearful', 'angry', 'calm', 'happy', 'fearful', 'fearful', 'angry',
       'fearful', 'fearful', 'calm', 'happy', 'angry', 'fearful',
       'fearful', 'fearful', 'fearful', 'happy', 'fearful', 'happy',
       'happy', 'angry', 'calm', 'calm', 'angry', 'calm', 'calm', 'happy',
       'fearful', 'calm', 'calm', 'fearful', 'calm', 'fearful', 'happy',
       'calm', 'happy', 'angry', 'angry', 'angry', 'happy', 'angry',
       'angry', 'fearful', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'happy', 'calm', 'fearful', 'calm', 'calm', 'happy', 'fearful',
       'happy', 'angry', 'angry', 'angry', 'angry', 'angry', 'happy',
       'angry', 'happy', 'calm', 'fearful', 'fearful', 'calm', 'happy',
       'calm', 'fearful', 'angry', 'fearful', 'fearful', 'fearful',
       'angry', 'fearful', 'fearful', 'calm', 'happy', 'fearful', 'angry',
   

In [27]:
# calculate the accuracy 
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 81.25%


### Save the model using pickle 

In [28]:
import pickle

In [29]:
filename = "Speech_emotion"
pickle.dump(model,open(filename,'wb'))

In [30]:
load_model = pickle.load(open(filename,"rb"))
print ("Accurcy is : " ,(load_model.score(x_test,y_test))*100)

Accurcy is :  81.25
